In [41]:
%matplotlib inline

In [42]:
import matplotlib.pyplot as plt

In [43]:
import geopandas
import os
import folium
from folium.plugins import MarkerCluster
from datetime import datetime

In [44]:
shp_file = r'D:\Dev\APS\aps\data\satskred\AvalDet_20190101_051134_ref_20181226_trno_095_VV\AvalDet_20190101_051134_ref_20181226_trno_095_VV.shp\AvalDet_20190101_051134_ref_20181226_trno_095_VV.shp'
gdf = geopandas.read_file(shp_file)

metadata_ = os.path.split(shp_file)[1].split('.')[0].split('_')
print(metadata_)
act_time = datetime.strptime(f'{metadata_[1]}T{metadata_[2]}', '%Y%m%dT%H%M%S') 
ref_date =  datetime.strptime(metadata_[4], '%Y%m%d')
orb = metadata_[6]
pol = metadata_[7]
print(ref_date, act_time)
gdf['activity_date'] = act_time
gdf['reference_date'] = ref_date
gdf['orbit'] = orb 
gdf['polarisation'] = pol

gdf.head(5)
#gdf.crs = {'init': 'EPSG:32633'}
gdf.crs = {'init': 'epsg:4326'}  # WGS84
gdf_utm33 = gdf.to_crs({'init': 'epsg:32633'})  # UTM 33

['AvalDet', '20190101', '051134', 'ref', '20181226', 'trno', '095', 'VV']
2018-12-26 00:00:00 2019-01-01 05:11:34


In [45]:
#gdf_utm33.plot()

In [46]:
#print(gdf_utm33['geometry'][5])

In [47]:
#print(gdf['geometry'].area)
#print(gdf['geometry'].centroid[:].x.mean())

In [48]:
#plt.hist(gdf_utm33.geometry.area)# / 1e6)  # area in sq.km

In [49]:
#gdf.columns

In [50]:
def show_on_map(gdf):

    gdf_wgs84 = gdf.to_crs({'init': 'epsg:4326'})  # make sure the projection is WGS84

    #
    lon = gdf_wgs84['geometry'].centroid[:].x.mean()
    lat = gdf_wgs84['geometry'].centroid[:].y.mean()
    zs = 9
    m = folium.Map(location=[lat, lon], zoom_start=zs, tiles="OpenStreetMap")

    topo4 = folium.raster_layers.WmsTileLayer(url="http://openwms.statkart.no/skwms1/wms.topo4?",
                                         layers="topo4_WMS", transparent=True, fmt="image/jpeg",
                                         name="Topo4", minZoom= "11")
    topo4.add_to(m)

    """
    Can request WMS data as geojson
    e.g.
    https://gis3.nve.no/map/rest/services/SkredSnoAktR/MapServer/2/query?where=1%3D1&text=&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&relationParam=&outFields=*&returnGeometry=true&returnTrueCurves=false&maxAllowableOffset=&geometryPrecision=&outSR=&returnIdsOnly=false&returnCountOnly=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&gdbVersion=&returnDistinctValues=false&resultOffset=&resultRecordCount=&queryByDistance=&returnExtentsOnly=false&datumTransformation=&parameterValues=&rangeValues=&f=geojson
    
    """

    kwargs = {"attribution": "NVE", "minZoom": "13"}
    # seems like a bug in folium prohibits that these parameters
    # are passed on to leaflet - can be set manually in the .html file

    bratthet = folium.raster_layers.WmsTileLayer(url="https://gis3.nve.no/map/services/Bratthet/MapServer/WmsServer?",
                                            layers="Bratthet_snoskred", transparent=True,
                                            fmt="image/png", name='Bratthet',
                                            **kwargs)
    bratthet.add_to(m)

    faresoner = folium.raster_layers.WmsTileLayer(url="https://gis3.nve.no/map/services/SkredSnoAktR/MapServer/WmsServer?",
                                         layers="Utlopsomrade", transparent=True, fmt="image/png",
                                         name="Avalanche run-out sone", **kwargs)
    faresoner.add_to(m)
    
    skredlop = folium.raster_layers.WmsTileLayer(url="https://www.vegvesen.no/kart/ogc/geodata_1_0/ows?",
                                            layers="Skredlop", transparent=True, fmt="image/png", name='SVV Skredløp', **kwargs)
    skredlop.add_to(m)

    #varslingsregioner = ImageOverlay(image='http://gis3.nve.no/map/rest/services/Mapservices/XgeoStatic/MapServer/export?dpi=96&transparent=true&format=png8&layers=show%3A25%2C26%2C29&bbox=-635878.2758445519%2C6520140.001693336%2C1616936.8964524597%2C7851779.998306664&bboxSR=32633&imageSR=32633&size=1663%2C983&f=image',
     #                                bounds=[[lat_min, lon_min], [lat_max, lon_max]])

    mc = MarkerCluster(name=f'Avalanches {gdf_wgs84.activity_date[0]} (location)').add_to(m)
    # for a in gdf_wgs84.geometry[:]:
    #     folium.Marker(location=[a.centroid.y, a.centroid.x]).add_to(mc)

    for a in gdf_wgs84.itertuples():
        # print(a, type(a))
        folium.Marker(location=[a.geometry.centroid.y, a.geometry.centroid.x],
                      popup=f'Date: {a.activity_date}<br>Orbit: {a.orbit}<br>Polarisation: {a.polarisation}').add_to(mc)

    folium.GeoJson(gdf_wgs84['geometry'], name=f'Avalanches {gdf_wgs84.activity_date[0]} (outlines)').add_to(m)

    # add layer control functionality to map
    folium.LayerControl().add_to(m)
    # # add lat/lon pop-up funcionality
    # folium.LatLngPopup().add_to(m)
    # # add measure functionality
    # plugins.MeasureControl().add_to(m)
    # # add drawing functionality
    # #folium.plugins.Draw().add_to(m)
    return m

In [51]:
m = show_on_map(gdf)

In [52]:
m

In [59]:
def load_gdf(shp_file):
    gdf = geopandas.read_file(shp_file)

    metadata_ = os.path.split(shp_file)[1].split('.')[0].split('_')
    print(metadata_)
    act_time = datetime.strptime(f'{metadata_[1]}T{metadata_[2]}', '%Y%m%dT%H%M%S') 
    ref_date =  datetime.strptime(metadata_[4], '%Y%m%d')
    orb = metadata_[6]
    pol = metadata_[7]
    print(ref_date, act_time)
    gdf['activity_date'] = act_time
    gdf['reference_date'] = ref_date
    gdf['orbit'] = orb 
    gdf['polarisation'] = pol

    gdf.head(5)
    #gdf.crs = {'init': 'EPSG:32633'}
    gdf.crs = {'init': 'epsg:4326'}  # WGS84
    gdf_utm33 = gdf.to_crs({'init': 'epsg:32633'})  # UTM 33
    return gdf

In [54]:
def add_to_map(m, shp_list):
    for shp_ in shp_list:
        gdf = load_gdf(shp_)
        gdf_wgs84 = gdf.to_crs({'init': 'epsg:4326'})
        mc = MarkerCluster(name=f'Avalanches {gdf_wgs84.activity_date[0]} (location)').add_to(m)

        for a in gdf_wgs84.itertuples():
            folium.Marker(location=[a.geometry.centroid.y, a.geometry.centroid.x],
                          popup=f'Date: {a.activity_date}<br>Orbit: {a.orbit}<br>Polarisation: {a.polarisation}').add_to(mc)

        folium.GeoJson(gdf_wgs84['geometry'], name=f'Avalanches {gdf_wgs84.activity_date[0]} (outlines)').add_to(m)
        
    return m

In [60]:
shp_list = [r'D:\Dev\APS\aps\data\satskred\AvalDet_20190102_050404_ref_20181227_trno_022_VV\AvalDet_20190102_050404_ref_20181227_trno_022_VV.shp\AvalDet_20190102_050404_ref_20181227_trno_022_VV.shp',
            r'D:\Dev\APS\aps\data\satskred\AvalDet_20190102_162349_ref_20181227_trno_029_VV\AvalDet_20190102_162349_ref_20181227_trno_029_VV.shp\AvalDet_20190102_162349_ref_20181227_trno_029_VV.shp',
            r'D:\Dev\APS\aps\data\satskred\AvalDet_20190103_161448_ref_20181228_trno_131_VV\AvalDet_20190103_161448_ref_20181228_trno_131_VV.shp\AvalDet_20190103_161448_ref_20181228_trno_131_VV.shp',
            r'D:\Dev\APS\aps\data\satskred\AvalDet_20190104_053610_ref_20181229_trno_139_VV\AvalDet_20190104_053610_ref_20181229_trno_139_VV.shp\AvalDet_20190104_053610_ref_20181229_trno_139_VV.shp',
            r'D:\Dev\APS\aps\data\satskred\AvalDet_20190104_160728_ref_20181229_trno_058_VV\AvalDet_20190104_160728_ref_20181229_trno_058_VV.shp\AvalDet_20190104_160728_ref_20181229_trno_058_VV.shp',
            r'D:\Dev\APS\aps\data\satskred\AvalDet_20190105_052837_ref_20181230_trno_066_VV\AvalDet_20190105_052837_ref_20181230_trno_066_VV.shp\AvalDet_20190105_052837_ref_20181230_trno_066_VV.shp',
            r'D:\Dev\APS\aps\data\satskred\AvalDet_20190105_155831_ref_20181230_trno_160_VV\AvalDet_20190105_155831_ref_20181230_trno_160_VV.shp\AvalDet_20190105_155831_ref_20181230_trno_160_VV.shp',
            r'D:\Dev\APS\aps\data\satskred\AvalDet_20190106_051945_ref_20181231_trno_168_VV\AvalDet_20190106_051945_ref_20181231_trno_168_VV.shp\AvalDet_20190106_051945_ref_20181231_trno_168_VV.shp'
           ]

In [63]:
def show_on_map(shp_list):
    
    gdf_list = [load_gdf(shp_) for shp_ in shp_list]
    
    gdf_wgs84 = gdf_list[0].to_crs({'init': 'epsg:4326'})  # make sure the projection is WGS84

    #
    lon = gdf_wgs84['geometry'].centroid[:].x.mean()
    lat = gdf_wgs84['geometry'].centroid[:].y.mean()
    zs = 9
    m = folium.Map(location=[lat, lon], zoom_start=zs, tiles="OpenStreetMap")

    topo4 = folium.raster_layers.WmsTileLayer(url="http://openwms.statkart.no/skwms1/wms.topo4?",
                                         layers="topo4_WMS", transparent=True, fmt="image/jpeg",
                                         name="Topo4", minZoom= "11")
    topo4.add_to(m)

    """
    Can request WMS data as geojson
    e.g.
    https://gis3.nve.no/map/rest/services/SkredSnoAktR/MapServer/2/query?where=1%3D1&text=&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&relationParam=&outFields=*&returnGeometry=true&returnTrueCurves=false&maxAllowableOffset=&geometryPrecision=&outSR=&returnIdsOnly=false&returnCountOnly=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&gdbVersion=&returnDistinctValues=false&resultOffset=&resultRecordCount=&queryByDistance=&returnExtentsOnly=false&datumTransformation=&parameterValues=&rangeValues=&f=geojson
    
    """

    kwargs = {"attribution": "NVE", "minZoom": "13"}
    # seems like a bug in folium prohibits that these parameters
    # are passed on to leaflet - can be set manually in the .html file

    bratthet = folium.raster_layers.WmsTileLayer(url="https://gis3.nve.no/map/services/Bratthet/MapServer/WmsServer?",
                                            layers="Bratthet_snoskred", transparent=True,
                                            fmt="image/png", name='Bratthet',
                                            **kwargs)
    bratthet.add_to(m)

    faresoner = folium.raster_layers.WmsTileLayer(url="https://gis3.nve.no/map/services/SkredSnoAktR/MapServer/WmsServer?",
                                         layers="Utlopsomrade", transparent=True, fmt="image/png",
                                         name="Avalanche run-out sone", **kwargs)
    faresoner.add_to(m)
    
    skredlop = folium.raster_layers.WmsTileLayer(url="https://www.vegvesen.no/kart/ogc/geodata_1_0/ows?",
                                            layers="Skredlop", transparent=True, fmt="image/png", name='SVV Skredløp', **kwargs)
    skredlop.add_to(m)

    #varslingsregioner = ImageOverlay(image='http://gis3.nve.no/map/rest/services/Mapservices/XgeoStatic/MapServer/export?dpi=96&transparent=true&format=png8&layers=show%3A25%2C26%2C29&bbox=-635878.2758445519%2C6520140.001693336%2C1616936.8964524597%2C7851779.998306664&bboxSR=32633&imageSR=32633&size=1663%2C983&f=image',
     #                                bounds=[[lat_min, lon_min], [lat_max, lon_max]])
    
    for gdf_ in gdf_list[1:]:
        gdf_wgs84 = gdf_.to_crs({'init': 'epsg:4326'}) 
        mc = MarkerCluster(name=f'Avalanches {gdf_wgs84.activity_date[0]} (location)').add_to(m)
        # for a in gdf_wgs84.geometry[:]:
        #     folium.Marker(location=[a.centroid.y, a.centroid.x]).add_to(mc)

        for a in gdf_wgs84.itertuples():
            # print(a, type(a))
            folium.Marker(location=[a.geometry.centroid.y, a.geometry.centroid.x],
                          popup=f'Date: {a.activity_date}<br>Orbit: {a.orbit}<br>Polarisation: {a.polarisation}').add_to(mc)

        folium.GeoJson(gdf_wgs84['geometry'], name=f'Avalanches {gdf_wgs84.activity_date[0]} (outlines)').add_to(m)

    # add layer control functionality to map
    folium.LayerControl().add_to(m)
    # # add lat/lon pop-up funcionality
    # folium.LatLngPopup().add_to(m)
    # # add measure functionality
    # plugins.MeasureControl().add_to(m)
    # # add drawing functionality
    # #folium.plugins.Draw().add_to(m)
    return m

In [64]:
m = show_on_map(shp_list)

['AvalDet', '20190102', '050404', 'ref', '20181227', 'trno', '022', 'VV']
2018-12-27 00:00:00 2019-01-02 05:04:04
['AvalDet', '20190102', '162349', 'ref', '20181227', 'trno', '029', 'VV']
2018-12-27 00:00:00 2019-01-02 16:23:49
['AvalDet', '20190103', '161448', 'ref', '20181228', 'trno', '131', 'VV']
2018-12-28 00:00:00 2019-01-03 16:14:48
['AvalDet', '20190104', '053610', 'ref', '20181229', 'trno', '139', 'VV']
2018-12-29 00:00:00 2019-01-04 05:36:10
['AvalDet', '20190104', '160728', 'ref', '20181229', 'trno', '058', 'VV']
2018-12-29 00:00:00 2019-01-04 16:07:28
['AvalDet', '20190105', '052837', 'ref', '20181230', 'trno', '066', 'VV']
2018-12-30 00:00:00 2019-01-05 05:28:37
['AvalDet', '20190105', '155831', 'ref', '20181230', 'trno', '160', 'VV']
2018-12-30 00:00:00 2019-01-05 15:58:31
['AvalDet', '20190106', '051945', 'ref', '20181231', 'trno', '168', 'VV']
2018-12-31 00:00:00 2019-01-06 05:19:45


In [65]:
m

In [67]:
m.save('aval_activity_indre_troms_01jan_06jan_2019.html')